In [ ]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import sklearn
import plotly.express as px
from pandas import Period
from scipy import stats
import re

## Dataset Clean

In [ ]:
ameco_1623 = pd.read_csv('/content/drive/MyDrive/Capstone Data/ameco_data.csv',low_memory=False)
df = ameco_1623.copy()

In [ ]:
df = df[['X.','Customer.No.','Customer.Name','Customer.Category',
                        'Ship.To.State','Invoice.Date','Strikeforce.Flag','Stock.Status',
                        'Item.No.','Item.Group','Product.Classification',
                        'UoM.Code','Quantity','Inventory.Cost','Price',
                        'DPM.Factor','DPM.Price']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155850 entries, 0 to 1155849
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   X.                      1155850 non-null  object 
 1   Customer.No.            1155850 non-null  object 
 2   Customer.Name           1155835 non-null  object 
 3   Customer.Category       590461 non-null   object 
 4   Ship.To.State           571691 non-null   object 
 5   Invoice.Date            1155850 non-null  object 
 6   Strikeforce.Flag        572301 non-null   object 
 7   Stock.Status            691242 non-null   object 
 8   Item.No.                1155840 non-null  object 
 9   Item.Group              572291 non-null   object 
 10  Product.Classification  572290 non-null   object 
 11  UoM.Code                1155835 non-null  object 
 12  Quantity                1155850 non-null  object 
 13  Inventory.Cost          1155847 non-null  object 
 14  Pr

In [ ]:
numerical_columns = ['Inventory.Cost','Price','Quantity']
df[numerical_columns] = df[numerical_columns].replace({',': ''}, regex=True).astype(float)
df = df[df['Price'] > 0]

### Customer Category Remap

#### Make the Customer Number consistent

In [ ]:
df['Customer.Name'] = df['Customer.Name'].str.replace(r'[^\w\s]+', '')

<ipython-input-24-ca9b5d3313d0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Customer.Name'] = df['Customer.Name'].str.replace(r'[^\w\s]+', '')


In [ ]:
df['Customer.Name'] = df['Customer.Name'].map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
df['Customer.Name']

0                             BEN HUR
1                 CUSTOM CONVEYOR INC
2                 CUSTOM CONVEYOR INC
3                 CUSTOM CONVEYOR INC
18                       KENNEDY TANK
                      ...            
1155845    ADS MANUFACTURING OHIO LLC
1155846    ADS MANUFACTURING OHIO LLC
1155847    ADS MANUFACTURING OHIO LLC
1155848    ADS MANUFACTURING OHIO LLC
1155849    ADS MANUFACTURING OHIO LLC
Name: Customer.Name, Length: 1134769, dtype: object

In [ ]:
def map_customer_no(row):
    if len(row['Customer.No.']) == 6 and 'PRE' in row['X.']:
        return 'C' + row['Customer.No.']
    elif len(row['Customer.No.']) == 5 and 'PRE' in row['X.']:
        return 'C0' + row['Customer.No.']
    elif len(row['Customer.No.']) == 4 and 'PRE' in row['X.']:
        return 'C00' + row['Customer.No.']
    elif len(row['Customer.No.']) == 3 and 'PRE' in row['X.']:
        return 'C000' + row['Customer.No.']
    elif len(row['Customer.No.']) == 2 and 'PRE' in row['X.']:
        return 'C0000' + row['Customer.No.']
    elif len(row['Customer.No.']) == 1 and 'PRE' in row['X.']:
        return 'C00000' + row['Customer.No.']
    else:
        return row['Customer.No.']

In [ ]:
df['Customer.No.'] = df.apply(map_customer_no, axis=1)

In [ ]:
print(f"The number of unique customer numberis now {df['Customer.No.'].nunique()}.")
print(f"The number of unique customer name is {df['Customer.Name'].nunique()}.")

The number of unique customer numberis now 2084.
The number of unique customer name is 2123.


In [ ]:
customer_pair = df[['Customer.No.','Customer.Name']]
customer_pair = customer_pair.groupby('Customer.No.')['Customer.Name'].nunique().reset_index().rename(columns={'Customer.Name':'Count'})

In [ ]:
customer_pair[customer_pair['Count']!=1]

,Customer.No.,Count
13,C000109,2
45,C000449,2
63,C000612,2
78,C000767,2
109,C001072,2
...,...,...
1681,C109882,2
1683,C109887,2
1692,C109902,2
1694,C109905,2


In [ ]:
df[df['Customer.No.']=='C001072'].sample(5)

,X.,Customer.No.,Customer.Name,Customer.Category,Ship.To.State,Invoice.Date,Strikeforce.Flag,Stock.Status,Item.No.,Item.Group,Product.Classification,UoM.Code,Quantity,Inventory.Cost,Price,DPM.Factor,DPM.Price
99064,PRE 99065,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ING 131-12,NaN,NaN,EA,1.0,5.240,6.00,NaN,NaN
91386,PRE 91387,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,S,ENE RC102K,NaN,NaN,EA,1.0,19.401,36.04,NaN,39.6
21061,PRE 21062,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ENE M783055,NaN,NaN,EA,1.0,3.620,6.00,NaN,NaN
311235,PRE 311236,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2018,NaN,N,GEC 202672-A,NaN,NaN,EA,1.0,480.380,620.00,NaN,650.5
99065,PRE 99066,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ING 244-TK2,NaN,NaN,EA,1.0,22.797,32.00,NaN,NaN


#### read customer category

In [ ]:
customer_df = pd.read_excel('/content/drive/MyDrive/Capstone Data/20240312 Customer Price List Assignment.xlsx',usecols=[0,1,2,3,4]).copy()

In [ ]:
customer_df.sample(10)

,#,Customer No.,Customer Name,DPM Exempt,Price List Name
1898,1899,C109171,MAGNOLIA PARTS INC.,N,Good
1285,1286,C107136,SUNDT CONSTRUCTION,N,Good
958,959,C104716,NUOVA PATAVIUM,N,Good
1726,1727,C108859,inactive SHAMOKIN DAM CONSTRUCTORS LLC,N,Good
2045,2046,C109345,CURTIS & SONS CONSTRUCTION INC,N,Good
2732,2733,C110134,INACTIVE Big Sky Construction,N,Good
1316,1317,C107294,WISCONSIN OPERATING ENG.TRAIN,N,Good
1964,1965,C109249,SKANSKA BURNS-MCDONNELL,N,Good
1113,1114,C105921,VALLEY RURAL UTILITY CO,N,Good
2853,2854,C110255,California Boiler DBA Oregon Boiler,N,Good


In [ ]:
customer_df['Customer No.'].nunique()

3595

In [ ]:
customer_df[customer_df['Customer No.'] == 'C001072']

,#,Customer No.,Customer Name,DPM Exempt,Price List Name
128,129,C001072,APPLIED INDUSTRIAL TECHNOLOGIES,N,Good


In [ ]:
customer_df['Customer No.'].unique()

array(['C000002', 'C000014', 'C000021', ..., 'C111271', 'C111272',
       'C180650'], dtype=object)

In [ ]:
no_list_1 = list(customer_df['Customer No.'].unique())
no_list_2 = list(df['Customer.No.'].unique())
print(f'Length of the updated file is {len(no_list_1)}')
print(f'Length of the original file is {len(no_list_2)}')
no_list_2 in no_list_1

Length of the updated file is 3595
Length of the original file is 2084


False

In [ ]:
customer_dict = dict(zip(customer_df['Customer No.'],customer_df['Customer Name']))

In [ ]:
def map_customer_name(row):
  if row['Customer.No.'] in customer_dict.keys():
    return customer_dict.get(row['Customer.No.'])
  else:
    return row['Customer.Name']

In [ ]:
df['customer_name'] = df.apply(map_customer_name, axis=1)

#### Map the customer category

In [ ]:
category_dict = dict(zip(customer_df['Customer No.'],customer_df['Price List Name']))

In [ ]:
def map_category(row):
  if row['Customer.No.'] in category_dict.keys():
    return category_dict.get(row['Customer.No.'])
  else:
    return row['Customer.Category']

In [ ]:
df['customer_category'] = df.apply(map_category, axis=1)

In [ ]:
df['customer_category'].value_counts()

Good            611198
Better          333080
Best            170152
Average Cost     17723
List Price        1399
M1                 464
P4                 290
P1                 268
P3                  90
P2                  66
M4                  29
P5                  10
Name: customer_category, dtype: int64

Check whether the customer belongs to a category consistently

> Unfortunately there are about 1507 has inconsistent customer categories.

In [ ]:
df[(df['Customer.Category'] != df['customer_category'])&(~df['Customer.Category'].isnull())]['Customer.No.'].nunique()

1507

#### Mark AMECO sub companies

In [ ]:
def mark_own_company(row):
  if row['customer_name'] in [
    "F&M MAFCO, INC",
    "CHRISTIANSTED EQUIPMENT, LTD.",
    "F&M MAFCO LLC",
    "EQUIPMENTSHARE.COM INC",
    "AMECO IC",
    "AMECO CANADA IC",
    "AMECO IC - VISTRA",
    "AMECO SERVICES INC (TRANSMOUNTAIN)"]:
    return True
  else:
    return False


In [ ]:
df['Own'] = df.apply(mark_own_company,axis=1)

In [ ]:
df['Own'].value_counts()

False    1102450
True       32319
Name: Own, dtype: int64

### Transform Datetime

In [ ]:
## date time
df['Invoice.Date'] = pd.to_datetime(df['Invoice.Date'])
df['Year'] = df['Invoice.Date'].dt.year
df['Month'] = df['Invoice.Date'].dt.month
df['Quarter'] = df['Invoice.Date'].dt.to_period('Q')

In [ ]:
def map_quarter(row):
  if row['Year'] == 2016:
    return row['Quarter']
  elif row ['Year'] == 2020:
    return Period('2019Q1', freq='Q-DEC')
  else:
    return row['Quarter'] - 4
df['Prev_Quarter'] = df.apply(map_quarter, axis=1)
df['Prev_Quarter'] = df['Prev_Quarter'].apply(lambda x: Period(x, freq='Q-DEC'))

### Flag

In [ ]:
## Flag mapping
def stock_flag(row):
    if row['Stock.Status'] == 'S' or row['Stock.Status'] == 'Stock':
        return 'Y'
    elif row['Stock.Status'] == 'N' or row['Stock.Status'] == 'Non Stock':
        return 'N'
    else:
      if row['Strikeforce.Flag'] == 'Y':
        return 'Y'
      elif row['Strikeforce.Flag'] == 'N':
        return 'N'
      else:
        return 'Y'

df['Stock'] = df.apply(stock_flag, axis=1)

In [ ]:
df.columns

Index(['X.', 'Customer.No.', 'Customer.Name', 'Customer.Category',
       'Ship.To.State', 'Invoice.Date', 'Strikeforce.Flag', 'Stock.Status',
       'Item.No.', 'Item.Group', 'Product.Classification', 'UoM.Code',
       'Quantity', 'Inventory.Cost', 'Price', 'DPM.Factor', 'DPM.Price',
       'customer_name', 'customer_category', 'Own', 'Year', 'Month', 'Quarter',
       'Prev_Quarter', 'Stock'],
      dtype='object')

In [ ]:
df.drop(columns={'Strikeforce.Flag','Stock.Status'},inplace=True)

### Item group & Product Classification

In [ ]:
## Item group and product classification
def fill_missing_most_frequent(group):
    mode_values = group.mode()
    if not mode_values.empty:
        mode_value = mode_values.iloc[0]
        return group.fillna(mode_value)
    else:
        return group

df['Product.Classification'] = df.groupby('Item.No.')['Product.Classification'].transform(fill_missing_most_frequent)
df['Item.Group'] = df.groupby('Item.No.')['Item.Group'].transform(fill_missing_most_frequent)

### Final Dataset

In [ ]:
df.drop(columns=['X.','Customer.Name'],inplace=True)

In [ ]:
df.to_csv('/content/drive/MyDrive/Capstone Data/updated_customer_dataset.csv',index=False)

In [ ]:
df

,Customer.No.,Customer.Category,Ship.To.State,Invoice.Date,Item.No.,Item.Group,Product.Classification,UoM.Code,Quantity,Inventory.Cost,...,DPM.Factor,DPM.Price,customer_name,customer_category,Own,Year,Month,Quarter,Prev_Quarter,Stock
0,C002309,P3,NaN,2016-01-01,KCP 14967,NaN,NaN,EA,6.0,31.768,...,NaN,41.46,BEN HUR CONSTRUCTION CO.,Better,False,2016,1,2016Q1,2016Q1,N
1,C000263,P3,NaN,2016-01-01,KNA 977-2PK,Const Supplies CS/SD,HANBOX,PKG,1.0,73.373,...,NaN,0,CUSTOM CONVEYOR INC,P3,False,2016,1,2016Q1,2016Q1,N
2,C000263,P3,NaN,2016-01-01,WES FA-30,Welding Equip WS/SD,APPFLA,EA,4.0,34.930,...,NaN,0,CUSTOM CONVEYOR INC,P3,False,2016,1,2016Q1,2016Q1,Y
3,C000263,P3,NaN,2016-01-01,PRO J07520,Const Supplies CS/SD,HANSOC,EA,3.0,11.584,...,NaN,18.8604,CUSTOM CONVEYOR INC,P3,False,2016,1,2016Q1,2016Q1,N
18,C001501,P3,NaN,2016-01-01,LIN ED010216,Welding Equip WS/SD,FILELM,LBS,150.0,2.014,...,NaN,0,KENNEDY TANK,Good,False,2016,1,2016Q1,2016Q1,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155845,C109148,NaN,OH,2021-06-16,GEC 713SUCB-M,Const Supplies CS/SD,SAFGLO,PKG12,1.0,7.680,...,0.0,0,ADS MANUFACTURING OHIO LLC,Better,False,2021,6,2021Q2,2020Q2,N
1155846,C109148,NaN,OH,2021-06-16,BER QT3-45,Welding Equip WS/SD,MIGPAR,EA,2.0,34.520,...,0.0,0,ADS MANUFACTURING OHIO LLC,Better,False,2021,6,2021Q2,2020Q2,N
1155847,C109148,NaN,OH,2021-06-16,LAK CTL412-3X,Const Supplies CS/SD,SAFCOV,EA,25.0,2.890,...,0.0,0,ADS MANUFACTURING OHIO LLC,Better,False,2021,6,2021Q2,2020Q2,N
1155848,C109148,NaN,OH,2021-06-16,MIL 216326,Welding Equip WS/SD,APPPRT,EA,10.0,1.600,...,0.0,0,ADS MANUFACTURING OHIO LLC,Better,False,2021,6,2021Q2,2020Q2,N


### Prepare for Profit Margin Dataset

In [ ]:
df.columns

Index(['Customer.No.', 'Customer.Category', 'Ship.To.State', 'Invoice.Date',
       'Item.No.', 'Item.Group', 'Product.Classification', 'UoM.Code',
       'Quantity', 'Inventory.Cost', 'Price', 'DPM.Factor', 'DPM.Price',
       'customer_name', 'customer_category', 'Own', 'Year', 'Month', 'Quarter',
       'Prev_Quarter', 'Stock'],
      dtype='object')

In [ ]:
profit_margin_df = df[['customer_name', 'customer_category', 'Own','Year',
                      'Month', 'Quarter', 'Prev_Quarter','Invoice.Date',
                      'Item.Group', 'Product.Classification',
                      'UoM.Code', 'Quantity', 'Inventory.Cost', 'Price',
                       'Ship.To.State', 'Stock']]

In [ ]:
profit_margin_df = profit_margin_df[(profit_margin_df['Year'] >=2020) &(profit_margin_df['Year'] <=2023)]

In [ ]:
profit_margin_df.rename(columns={'Invoice.Date':'Date','Item.Group':'Group','Product.Classification':'Product_Class','UoM.Code':'Unit','Inventory.Cost':'Cost','Ship.To.State':'State'},inplace=True)

<ipython-input-66-b2ffe28618d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit_margin_df.rename(columns={'Invoice.Date':'Date','Item.Group':'Group','Product.Classification':'Product_Class','UoM.Code':'Unit','Inventory.Cost':'Cost','Ship.To.State':'State'},inplace=True)


In [ ]:
profit_margin_df['Profit_Margin'] = 100*(profit_margin_df['Price'] - profit_margin_df['Cost'])/profit_margin_df['Price']

<ipython-input-67-2034cc6f4d8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profit_margin_df['Profit_Margin'] = 100*(profit_margin_df['Price'] - profit_margin_df['Cost'])/profit_margin_df['Price']


In [ ]:
profit_margin_df.to_csv('/content/drive/MyDrive/Capstone Data/pm_dataset.csv',index=False)

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Capstone Data/pm_dataset.csv')

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568081 entries, 0 to 568080
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_name      568081 non-null  object 
 1   customer_category  568081 non-null  object 
 2   Own                568081 non-null  bool   
 3   Year               568081 non-null  int64  
 4   Month              568081 non-null  int64  
 5   Quarter            568081 non-null  object 
 6   Prev_Quarter       568081 non-null  object 
 7   Date               568081 non-null  object 
 8   Group              567172 non-null  object 
 9   Product_Class      567171 non-null  object 
 10  Unit               568077 non-null  object 
 11  Quantity           568081 non-null  float64
 12  Cost               568081 non-null  float64
 13  Price              568081 non-null  float64
 14  State              554163 non-null  object 
 15  Stock              568081 non-null  object 
 16  Pr

In [ ]:
test_df.sample(10)

,customer_name,customer_category,Own,Year,Month,Quarter,Prev_Quarter,Date,Group,Product_Class,Unit,Quantity,Cost,Price,State,Stock,Profit_Margin
366700,INACTIVE KIEWIT POWER CONST. WHS 152,Good,False,2020,9,2020Q3,2019Q1,2020-09-16,Const Supplies CS/SD,CUTCIR,EA,6.0,1.08,1.42,IL,Y,23.943662
516406,WOOD-MIZER LLC WHSE 500,Better,False,2021,4,2021Q2,2020Q2,2021-04-01,Welding Equip WS/SD,MIGNOZ,EA,1.0,7.83,8.35,IN,Y,6.227545
168581,INACTIVE KIEWIT POWER WHSE 176,Good,False,2022,2,2022Q1,2021Q1,2022-02-11,Const Supplies CS/SD,CUTREA,EA,2.0,55.38,78.29,IL,Y,29.262997
220180,ADVANCED FIELD SERVICES,Good,False,2022,10,2022Q4,2021Q4,2022-10-19,Welding Equip WS/SD,FILELS,LBS,10.0,11.24,13.87,WV,Y,18.961788
17761,INACTIVE KIEWIT POWER WHS 146,Good,False,2021,6,2021Q2,2020Q2,2021-06-22,Welding Equip WS/SD,TIGGAS,EA,2.0,1.90,2.56,MI,Y,25.781250
89913,F&M MAFCO LLC,Average Cost,True,2020,2,2020Q1,2019Q1,2020-02-21,Const Supplies CS/SD,SAFGLO,PR,35.0,6.47,6.47,BC,N,0.000000
282528,ADS MANUFACTURING OHIO LLC,Better,False,2023,8,2023Q3,2022Q3,2023-08-17,Const Supplies CS/SD,SAFGLO,PKG100,2.0,7.58,15.10,OH,Y,49.801325
111925,WOOD-MIZER LLC WHSE 500,Better,False,2021,10,2021Q4,2020Q4,2021-10-04,Welding Equip WS/SD,WEMACC,EA,2.0,4.37,6.07,IN,Y,28.006590
30437,INACTIVE B&W/WHSE 173,Good,False,2021,6,2021Q2,2020Q2,2021-06-28,Welding Equip WS/SD,TIGTUN,PKG10,2.0,13.70,18.07,MO,Y,24.183730
138129,TRAYLOR INDUSTRIAL,Good,False,2021,11,2021Q4,2020Q4,2021-11-11,Const Supplies CS/SD,MAIPOL,EA,2.0,62.80,83.79,IN,Y,25.050722
